In [1]:
### https://github.com/hhursev/recipe-scrapers
from recipe_scrapers import scrape_me
import time
import pandas as pd
import numpy as np
from fractions import Fraction
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

## give the url as a string, it can be url from any site listed below
#scraper = scrape_me('https://www.allrecipes.com/recipe/220943/chef-johns-buttermilk-biscuits/')

In [2]:
#Links I get from a web app called "web scrapper" and All Recipe's and Food Networks Websites

all_recipes_scone_links = pd.read_csv('all_recipes_scone_links.csv')
all_recipes_biscuit_links = pd.read_csv('all_recipes_biscuit_links.csv')
food_network_scone_links = pd.read_csv('food_network_scones.csv')
food_network_biscuit_links = pd.read_csv('food_network_biscuits.csv')

In [3]:
#### Creating all recipes scone recipe array
recipes = []
for index, row in all_recipes_scone_links.iterrows():
    x = scrape_me(all_recipes_scone_links.iloc[index][0])
    row_data = ['Scones',
                x.title(),
                x.category(),
                x.canonical_url(),
                x.site_name(),
                x.ingredients(),
                x.instructions(),
                x.nutrients(),
                x.prep_time(),
                x.cook_time(),
                x.total_time(),
                x.yields()]
    recipes.append(row_data)
    time.sleep(.2)

In [4]:
#### Creating food network scone recipe array
for index, row in food_network_scone_links.iterrows():
    x = scrape_me(food_network_scone_links.iloc[index][0])
    try:
        a = x.total_time()
    except: 
        a = 0
    
    
    row_data = ['Scones',
                x.title(),
                'Scones',
                x.canonical_url(),
                x.site_name(),
                x.ingredients(),
                x.instructions(),
                x.nutrients(),
                0,
                0,
                a,
                x.yields()]
    recipes.append(row_data)
    time.sleep(.2)

In [5]:
#### Creating all recipes Biscuit Recipe Array
for index, row in all_recipes_biscuit_links.iterrows():
    x = scrape_me(all_recipes_biscuit_links.iloc[index][0])
    row_data = ['Biscuit',
                x.title(),
                x.category(),
                x.canonical_url(),
                x.site_name(),
                x.ingredients(),
                x.instructions(),
                x.nutrients(),
                x.prep_time(),
                x.cook_time(),
                x.total_time(),
                x.yields()]
    recipes.append(row_data)
    time.sleep(.2)

In [ ]:
#### Creating food network Biscuit Recipe Array
for index, row in food_network_biscuit_links.iterrows():
    x = scrape_me(food_network_biscuit_links.iloc[index][0])
    try:
        a = x.total_time()
    except: 
        a = 0
    
    row_data = ['Biscuit',
                x.title(),
                'Biscuit',
                x.canonical_url(),
                x.site_name(),
                x.ingredients(),
                x.instructions(),
                x.nutrients(),
                0,
                0,
                a,
                x.yields()]
    recipes.append(row_data)
    time.sleep(.2)

In [ ]:
import pickle
with open('recipes.pkl', 'wb') as f:
    pickle.dump(recipes, f)

In [ ]:
import pickle
with open('recipes.pkl','rb')as f:
    recipes = pickle.load(f)


In [ ]:
#### Creating DataFrame
column_names = ['category',
                'title',
                'category_website',
                'url',
                'site_name',
                'ingredients',
                'instructions',
                'nutrients',
                'prep_time',
                'cook_time',
                'total_time',
                'yields']
recipes_df = pd.DataFrame(recipes, columns = column_names)

### Cleaning Data

In [ ]:
british_index = (recipes_df.loc[recipes_df['url']=='https://www.foodnetwork.com/recipes/grapefruit-and-pepper-meringue-tartlets-2109834']).index[0]
recipes_df = recipes_df.drop(index = british_index)
crumble_index = (recipes_df.loc[recipes_df['url']=='https://www.foodnetwork.com/recipes/biscuit-crumble-with-chocolate-and-raspberries-3415125']).index[0]
recipes_df = recipes_df.drop(index = crumble_index)
quail_index = (recipes_df.loc[recipes_df['url']=='https://www.foodnetwork.com/recipes/eggs-benedict-quail-eggs-spiced-biscuits-lemongrass-crab-fennel-salad-recipe-1918473']).index[0]
recipes_df = recipes_df.drop(index = quail_index)

I should come back and rewrite this after knowing some of the possible pitfalls in the foodnetwork dataset

In [ ]:
# Functions to clean up dataframe
def baking_temp(string):
    x = string.rfind(' degrees F')
    if x == -1:
        x = string.rfind(' degree F')
        if x==-1:
            x = string.rfind(' degree')
            if x==-1:
                return(0)
            else:
                return(int(string[x-3:x]))
        else:
            return(int(string[x-3:x]))
    else:
        return(int(string[x-3:x]))
    
def get_cals(dictionary):
    try:
        a = float(dictionary['calories'].strip(' calories').strip('cals'))
    except: 
        a = -1
    return(a)

def get_carbs(dictionary):
    try:
        a = float(dictionary['carbohydrateContent'].strip('grams').strip('g'))
    except: 
        a = -1
    return(a)

def get_chol(dictionary):
    
    if 'cholesterolContent' not in list(dictionary.keys()):
        return(-1)
    else:
        return(float(dictionary['cholesterolContent'].strip(' mg').strip('milligrams')))

def get_fat(dictionary):
    try:
        a = float(dictionary['fatContent'].strip(' grams'))
    except: 
        a = -1
    return(a)

def get_fiber(dictionary):
    if 'fiberContent' not in list(dictionary.keys()):
        return(-1)
    else:
        return(float(dictionary['fiberContent'].strip(' grams')))

def get_protein(dictionary):
    try:
        a = float(dictionary['proteinContent'].strip(' grams'))
    except: 
        a = -1
    return(a)

def get_satfat(dictionary):
    try:
        a = float(dictionary['saturatedFatContent'].strip(' grams'))
    except: 
        a = -1
    return(a)

def get_sodium(dictionary):
    try:
        a = float(dictionary['sodiumContent'].strip('milligrams').strip(' mg'))
    except: 
        a = -1
    return(a)

def get_sugar(dictionary):
    if 'sugarContent' not in list(dictionary.keys()):
        return(-1)
    else:
        return(float(dictionary['sugarContent'].strip(' grams')))
    
def get_servings(string):
    return(int(string.split(' ',1)[0]))

# trying to find how thick to roll 0 means the function didn't work
def thickness(string):
    x = string.find(' thick')
    if string[x-1] == 'h':
        return(float(Fraction(string[x-8:x].strip(' inch').strip('-'))))
    else:
        return(0)
    
def num_ingredients(listy):
    return(len(listy))

def num_steps(string):
    return(string.count('\n')+1)

def ingredient_amt(ingredient_list, item):
    amt = 0
    for x in ingredient_list:
        if item in x and 'stick' in x and 'sticky' not in x:
            nums = x.split(' stick')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'heaping' in nums:
                nums = nums.split('heaping')[0]
            if ' ' in nums:
                amt = (float(nums.split(' ')[0])+unicodedata.numeric(nums.split(' ')[1]))/2+amt
            elif nums.isdigit() == True:
                amt = (float(nums))/2 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))/2 + amt
        
        if item in x and 'quart' in x:
            nums = x.split(' quart')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if ' ' in nums:
                ratio = 4
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums))*4 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))*4 + amt
        
        if item in x and 'cup' in x:
            nums = x.split(' cup')[0].split('-cup')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'about' in nums:
                nums = nums.split('about ')[1]
            if 'About' in nums:
                nums = nums.split('About ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:  
                amt = float(nums.split(' ')[0])+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1])) + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])+amt
            elif nums.isdigit() == True:
                amt = (float(nums)) + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums)) + amt
                
        if item in x and 'tablespoon' in x:
            nums = x.split(' tablespoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if 'more' in nums:
                nums = nums.split(' more')[0]
            if ' ' in nums:
                ratio = (1/16)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums)/16) + amt
            else:
                ratio = (1/16)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
                
        if item in x and 'teaspoon' in x:
            nums = x.split(' teaspoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'and' in nums:
                nums = nums.split('and ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            
            if ' ' in nums:
                ratio = (1/48)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
                
            elif nums.isdigit() == True:
                amt = (float(nums)/48) + amt
            else:
                ratio = (1/48)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)/48) + amt
    return(amt)


def flour(ingredient_list):
    return(ingredient_amt(ingredient_list, 'flour'))

def sugar(ingredient_list):
    return(ingredient_amt(ingredient_list, 'sugar'))

def bakingsoda(ingredient_list):
    return(ingredient_amt(ingredient_list, 'baking soda'))

def bakingpowder(ingredient_list):
    return(ingredient_amt(ingredient_list, 'baking powder'))

#def salt(ingredient_list):
    return(ingredient_amt(ingredient_list, 'salt'))

def milk(ingredient_list):
    return(ingredient_amt(ingredient_list, ' milk'))

def buttermilk(ingredient_list):
    return(ingredient_amt(ingredient_list, 'buttermilk'))

def water(ingredient_list):
    return(ingredient_amt(ingredient_list, 'water'))

def sourcream(ingredient_list):
    return(ingredient_amt(ingredient_list, 'sour cream'))

def margarine(ingredient_list):
    return(ingredient_amt(ingredient_list, 'margarine'))

def shortening(ingredient_list):
    return(ingredient_amt(ingredient_list, 'shortening'))

def butter(ingredient_list):
    item = 'butter'
    amt = 0
    for x in ingredient_list:
        if item in x and 'stick' in x and 'sticky' not in x and 'milk' not in x and 'cup' not in x:
            nums = x.split(' stick')[0]
#            if 'to' in nums:
#                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'cold' in nums:
                nums = nums.split(' cold')[0]
            if ' ' in nums:
                ratio = (1/2)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                ratio = (1/2)
                amt = (float(nums)*ratio) + amt
            else:
                ratio = (1/2)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
        
        if item in x and 'quart' in x and 'milk' not in x:
            nums = x.split(' quart')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if ' ' in nums:
                ratio = 4
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums))*4 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))*4 + amt
        
        if item in x and 'cup' in x and 'milk' not in x:
            nums = x.split(' cup')[0].split('-cup')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'about' in nums:
                nums = nums.split('about ')[1]
            if 'About' in nums:
                nums = nums.split('About ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:  
                amt = float(nums.split(' ')[0])+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1])) + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])+amt
            elif nums.isdigit() == True:
                amt = (float(nums)) + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums)) + amt
                
        if item in x and 'tablespoon' in x and 'milk' not in x and 'stick' not in x:
            nums = x.split(' tablespoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if 'more' in nums:
                nums = nums.split(' more')[0]
            if ' ' in nums:
                ratio = (1/16)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums)/16) + amt
            else:
                ratio = (1/16)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
                
        if item in x and 'teaspoon' in x and 'milk' not in x:
            nums = x.split(' teaspoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'and' in nums:
                nums = nums.split('and ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            
            if ' ' in nums:
                ratio = (1/48)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
                
            elif nums.isdigit() == True:
                amt = (float(nums)/48) + amt
            else:
                ratio = (1/48)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)/48) + amt
    return(amt)

def cream(ingredient_list):
    item = 'cream'
    amt = 0
    for x in ingredient_list:
        if item in x and 'stick' in x and 'sticky' not in x and 'sour' not in x:
            nums = x.split(' stick')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'heaping' in nums:
                nums = nums.split('heaping')[0]
            if ' ' in nums:
                amt = (float(nums.split(' ')[0])+unicodedata.numeric(nums.split(' ')[1]))/2+amt
            elif nums.isdigit() == True:
                amt = (float(nums))/2 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))/2 + amt
        
        if item in x and 'quart' in x and 'sour' not in x:
            nums = x.split(' quart')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if ' ' in nums:
                ratio = 4
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums))*4 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))*4 + amt
        
        if item in x and 'cup' in x and 'sour' not in x:
            nums = x.split(' cup')[0].split('-cup')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'about' in nums:
                nums = nums.split('about ')[1]
            if 'About' in nums:
                nums = nums.split('About ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:  
                amt = float(nums.split(' ')[0])+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1])) + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])+amt
            elif nums.isdigit() == True:
                amt = (float(nums)) + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums)) + amt
                
        if item in x and 'tablespoon' in x and 'sour' not in x:
            nums = x.split(' tablespoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:
                ratio = (1/16)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums)/16) + amt
            else:
                ratio = (1/16)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
                
        if item in x and 'teaspoon' in x and 'sour' not in x:
            nums = x.split(' teaspoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'and' in nums:
                nums = nums.split('and ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:
                ratio = (1/48)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
                
            elif nums.isdigit() == True:
                amt = (float(nums)/48) + amt
            else:
                ratio = (1/48)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)/48) + amt
    return(amt)


def salt(ingredient_list):
    item = 'salt'
    amt = 0
    for x in ingredient_list:
        if item in x and 'stick' in x and 'sticky' not in x and 'butter' not in x:
            nums = x.split(' stick')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'heaping' in nums:
                nums = nums.split('heaping')[0]
            if ' ' in nums:
                ratio = (1/2)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums)*ratio) + amt
            else:
                ratio = (1/2)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
        
        if item in x and 'quart' in x and 'butter' not in x:
            nums = x.split(' quart')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if ' ' in nums:
                ratio = 4
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                amt = (float(nums))*4 + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums))*4 + amt
        
        if item in x and 'cup' in x and 'butter' not in x:
            nums = x.split(' cup')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:
                amt = float(nums.split(' ')[0])+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1])) + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])+amt
            elif nums.isdigit() == True:
                amt = (float(nums)) + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums)) + amt
                
        if item in x and 'tablespoon' in x and 'butter' not in x:
            nums = x.split(' tablespoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'with' in nums:
                nums = nums.split('with ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:
                ratio = (1/16)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
            elif nums.isdigit() == True:
                ratio = (1/16)
                amt = (float(nums)*ratio) + amt
            else:
                ratio = (1/16)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)*ratio) + amt
                
        if item in x and 'teaspoon' in x and 'butter' not in x:
            nums = x.split(' teaspoon')[0]
            if 'to' in nums:
                nums = nums.split('to ')[1]
            if 'plus' in nums:
                nums = nums.split('plus ')[1]
            if '(' in nums:
                nums = nums.split('(')[1]
            if 'and' in nums:
                nums = nums.split('and ')[1]
            if 'heaping' in nums:
                nums = nums.split(' heaping')[0]
            if 'level' in nums:
                nums = nums.split(' level')[0]
            if 'scant' in nums:
                nums = nums.split(' scant')[0]
            if ' ' in nums:
                ratio = (1/48)
                amt = float(nums.split(' ')[0])*ratio+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1]))*ratio + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])*ratio+amt
                
            elif nums.isdigit() == True:
                amt = (float(nums)/48) + amt
            else:
                ratio = (1/48)
                if '/' in nums:
                    amt = (float((nums).split('/')[0])/float((nums).split('/')[1]))*ratio + amt
                else:
                    amt = (unicodedata.numeric(nums)/48) + amt
    return(amt)

def eggs(ingredient_list):
    amt = 0
    for x in ingredient_list:
#        if 'egg' in x and 'cup' in x:
#            nums = x.split(' cup')[0]
#            if nums.isdigit is True:
#                egg_count = egg_count+float(nums)*5
#            if nums.isdigit is False:
#                egg_count = unicodedata.numeric(nums)*5 + egg_count
#       
        if 'egg' in x and 'nog' not in x and 'cup' not in x and 'ounce' not in x:
            nums = x.split(' egg')[0]
            if 'suggestion:' in nums:
                nums = '0'
            if 'to' in nums:
                nums = nums.split(' to')[0]
            if 'fried' in nums:
                nums = nums.split(' fried')[0]
            if 'hard-boiled' in nums:
                nums = nums.split(' hard-boiled')[0]
            if 'beaten' in nums:
                nums = nums.split(' beaten')[0]
            if 'whole' in nums:
                nums = nums.split(' whole')[0]
            if 'extra' in nums:
                nums = nums.split(' extra')[0]
            if 'large' in nums:
                nums = nums.split(' large')[0]
            if 'medium' in nums:
                nums = nums.split(' medium')[0]
            if 'small' in nums:
                nums = nums.split(' small')[0]
            if 'wash, ' in nums:
                nums = nums.split('wash, ')[1]
            if ' ' in nums:
                amt = float(nums.split(' ')[0])+ amt 
                if '/' in nums.split(' ')[1]:
                    amt = (float((nums.split(' ')[1]).split('/')[0])/float((nums.split(' ')[1]).split('/')[1])) + amt
                else:
                    amt = unicodedata.numeric(nums.split(' ')[1])+amt
            elif nums.isdigit() == True:
                amt = (float(nums)) + amt
            elif len(nums) == 1:
                amt = (unicodedata.numeric(nums)) + amt
            #egg_count = float(nums.split(' ')[0].split(' ')[0])
    return(amt)


In [ ]:
# applying those functions
recipes_df['baking_temp'] = recipes_df.instructions.apply(baking_temp)
recipes_df['calories'] = recipes_df.nutrients.apply(get_cals)
recipes_df['carbs'] = recipes_df.nutrients.apply(get_carbs)
recipes_df['chol'] = recipes_df.nutrients.apply(get_chol)
recipes_df['fat'] = recipes_df.nutrients.apply(get_fat)
recipes_df['fiber'] = recipes_df.nutrients.apply(get_fiber)
recipes_df['protein'] = recipes_df.nutrients.apply(get_protein)
recipes_df['satfat'] = recipes_df.nutrients.apply(get_satfat)
recipes_df['sodium'] = recipes_df.nutrients.apply(get_sodium)
recipes_df['sugar'] = recipes_df.nutrients.apply(get_sugar)
recipes_df['servings'] = recipes_df.yields.apply(get_servings)
#recipes_df['thickness'] = recipes_df.instructions.apply(thickness)
recipes_df['num_ingredients'] = recipes_df.ingredients.apply(num_ingredients)
recipes_df['num_steps'] = recipes_df.instructions.apply(num_steps)
recipes_df['cups_flour'] = recipes_df.ingredients.apply(flour)
recipes_df['cups_sugar'] = recipes_df.ingredients.apply(sugar)
recipes_df['baking_soda'] = recipes_df.ingredients.apply(bakingsoda)
recipes_df['baking_powder'] = recipes_df.ingredients.apply(bakingpowder)
recipes_df['salt'] = recipes_df.ingredients.apply(salt)
recipes_df['milk'] = recipes_df.ingredients.apply(milk)
recipes_df['cream'] = recipes_df.ingredients.apply(cream)
recipes_df['buttermilk'] = recipes_df.ingredients.apply(buttermilk)
recipes_df['water'] = recipes_df.ingredients.apply(water)
recipes_df['sour_cream'] = recipes_df.ingredients.apply(sourcream)
recipes_df['butter'] = recipes_df.ingredients.apply(butter)
recipes_df['margarine'] = recipes_df.ingredients.apply(margarine)
recipes_df['shortening'] = recipes_df.ingredients.apply(shortening)
recipes_df['eggs'] = recipes_df.ingredients.apply(eggs)

It looks like the food network data was particularly dirty. I'll need to compeletely rewrite the cleaning functions to try and tackle this so for now I'm going to focus on just the all recipe data. On top of this I have no nutrition data for Food Network

In [ ]:
allrecipes_df = recipes_df[recipes_df.site_name == 'Allrecipes']#.reset_index()

In [ ]:
limited_mask = ['category', 'servings','baking_temp','prep_time','cook_time',
       'num_ingredients', 'num_steps', 'cups_flour', 'cups_sugar',
       'baking_soda', 'baking_powder', 'salt', 'milk', 'buttermilk', 'water',
       'sour_cream', 'butter', 'margarine', 'shortening', 'eggs',
        'calories','sugar','carbs','chol','fat','fiber','protein','satfat','sodium']
allrec_df = allrecipes_df[limited_mask]

In [ ]:
more_limited_mask = ['url','category', 'servings','baking_temp','prep_time','cook_time',
       'num_ingredients', 'num_steps', 'cups_flour', 'cups_sugar',
       'baking_soda', 'baking_powder', 'salt', 'milk', 'buttermilk', 'water',
       'sour_cream', 'butter', 'margarine', 'shortening', 'eggs',
        'calories']
allrec_df = allrecipes_df[more_limited_mask]

In [ ]:
len(more_limited_mask)

In [ ]:
sns.pairplot(allrec_df,hue = 'category')

# Begin Modeling Phase

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_roc_curve
from sklearn import metrics

In [ ]:
def binary_scone_biscuit(x):
    if x == 'Biscuit':
        return(0)
    if x == 'Scones':
        return(1)

In [ ]:
allrec_df['binary_cat'] = allrec_df.category.apply(binary_scone_biscuit)

## Splitting Data

In [ ]:
X = allrec_df.drop(columns = ['category','binary_cat','url'])
y = allrec_df[['binary_cat','url']]

X_train_val, X_test, y_train_va11, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train1, y_val1 = train_test_split(X_train_val, y_train_val1, test_size=0.25, random_state=42)

y_train_val = y_train_val1.binary_cat
y_test = y_test1.binary_cat
y_train = y_train1.binary_cat
y_val = y_val1.binary_cat

### Scalling Data

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns = X_val.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)



## Decision Tree

Just out of curiosity I want to see what a Decision Tree on the training data would be

In [ ]:
dt = tree.DecisionTreeClassifier(max_depth = 3)
dt.fit(X_train,y_train)

In [ ]:
#tree.plot_tree(dt,fontsize = 10)
import graphviz 
dot_data = tree.export_graphviz(dt, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decision_tree") 

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None, 
                   feature_names=list(X_train.columns),  
                   class_names=['Scones','Biscuit'],  
                   filled=True, rounded=True,  
                   special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

## Logistic Regression

In [ ]:
logreg = LogisticRegression(random_state=0)

In [ ]:
logreg.fit(X_train_scaled,y_train)

In [ ]:
logreg.score(X_train_scaled,y_train)

In [ ]:
logreg.score(X_val_scaled,y_val)

In [ ]:
y_val_pred = logreg.predict(X_val_scaled)
precision_score(y_val,y_val_pred)

In [ ]:
y_val_pred = logreg.predict(X_val_scaled)
recall_score(y_val,y_val_pred)

In [ ]:
plot_roc_curve(logreg,X_train_scaled,y_train)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
plot_roc_curve(logreg,X_val_scaled,y_val)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
logreg_coeff = pd.DataFrame({'feature_name': logreg.feature_names_in_, 
                             'model_coefficient': logreg.coef_.transpose().flatten()})
logreg_coeff = logreg_coeff.sort_values(by = 'model_coefficient',ascending = False)

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="feature_name", y="model_coefficient", data=logreg_coeff)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

## Kernel Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
list(range(1,15))

In [ ]:
n_neighbors = list(range(1,20))
scores =[]
for x in n_neighbors:
    neigh = KNeighborsClassifier(n_neighbors = x)
    neigh.fit(X_train_scaled,y_train)
    scores.append(neigh.score(X_val_scaled,y_val))
plt.plot(n_neighbors,scores)
plt.xlabel('number of neighbors')
plt.ylabel('validation score')

In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(X_train_scaled,y_train)

In [ ]:
neigh.score(X_train_scaled,y_train)

In [ ]:
neigh.score(X_val_scaled,y_val)

In [ ]:
y_val_pred = neigh.predict(X_val_scaled)
precision_score(y_val,y_val_pred)

In [ ]:
y_val_pred = neigh.predict(X_val_scaled)
recall_score(y_val,y_val_pred)

In [ ]:
plot_roc_curve(neigh,X_train_scaled,y_train)

In [ ]:
plot_roc_curve(neigh,X_val_scaled,y_val)

## Random Forrest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
frst = RandomForestClassifier(n_estimators = 100, max_depth=3, random_state=0)

frst.fit(X_train_scaled, y_train)

In [ ]:
frst.score(X_train_scaled, y_train)

In [ ]:
frst.score(X_val_scaled,y_val)

In [ ]:
y_val_pred = frst.predict(X_val_scaled)
precision_score(y_val,y_val_pred)

In [ ]:
y_val_pred = frst.predict(X_val_scaled)
recall_score(y_val,y_val_pred)

In [ ]:
plot_roc_curve(frst,X_train_scaled,y_train)

In [ ]:
plot_roc_curve(frst,X_val_scaled,y_val)
auc = metrics.roc_auc_score(y_val,y_val_pred)
print(auc)
#### Not sure why these aren't matcching. I'm more inclined to believe the metrics.roc_auc_score method

## Plot ROC curves Together

In [ ]:
plt.figure()
fig, ax = plt.subplots() # or 
fig.patch.set_facecolor('none')
fig.patch.set_facecolor('none')

models = [
{
    'label': 'Logistic Regression',
    'model': logreg,
},
{
    'label': 'Random Forest',
    'model': frst,
},
{
    'label': 'K-Nearest Neighbors',
    'model': neigh,
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    #model.fit(X_train_scaled, y_train) # train the model
    y_pred=model.predict(X_val_scaled) # predict the test data
# Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_val, model.predict_proba(X_val_scaled)[:,1])
# Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_val,model.predict(X_val_scaled))
# Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Fall Out (False Positive Rate)')
plt.ylabel('Recall (True Positive Rate)')
plt.title('Receiver Operating Characteristic',fontsize = 15)
plt.legend(loc="lower right")
plt.show()   # Display
fig.savefig('filename.eps', format='eps')

 ## Logistic Regression with Lasso Penalty and Cross Validation

In [ ]:
scalercv = preprocessing.StandardScaler().fit(X_train_val)
X_train_val_scaled = pd.DataFrame(scalercv.transform(X_train_val), columns = X_train_val.columns)
X_test_scaled_cv = pd.DataFrame(scalercv.transform(X_test), columns = X_test.columns)


In [ ]:
len(scores)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
scores = []

C = np.logspace(-1.8,3,num = 500)
fig, ax = plt.subplots() # or 
fig.patch.set_facecolor('none')
ax.patch.set_facecolor('none')

for x in C:
    logregcv = LogisticRegressionCV(Cs = [x],cv = 5, random_state = 0,solver = 'liblinear', penalty = 'l1')
    logregcv.fit(X_train_val_scaled,y_train_val)
    scores.append(logregcv.score(X_train_val_scaled,y_train_val))   
    
plt.plot(np.log10(C),scores,linewidth = 3)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.xlabel('Log(C) Regularization Strength', fontsize = 15)
plt.ylabel('Accuracy', fontsize = 15)
plt.title('CV = 5 Lasso Grid Search', fontsize = 15)
fig.savefig('grid_search.pdf', format='eps',bbox_inches = 'tight',pad_inches = 1)

In [ ]:
logregcv = LogisticRegressionCV(Cs = [.1],cv = 5, random_state = 0,solver = 'liblinear', penalty = 'l1')
logregcv.fit(X_train_val_scaled,y_train_val)

In [ ]:
logreg_coeff = pd.DataFrame({'feature_name': logregcv.feature_names_in_, 
                             'model_coefficient': logregcv.coef_.transpose().flatten()})
logreg_coeff = logreg_coeff.sort_values(by = 'model_coefficient',ascending = False)

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="feature_name", y="model_coefficient", data=logreg_coeff)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

In [ ]:
logregcv.score(X_train_val_scaled,y_train_val)

In [ ]:
plot_roc_curve(logregcv,X_train_val_scaled,y_train_val)

In [ ]:
plot_roc_curve(logregcv,X_train_scaled,y_train)

#### Redoing Pairplots

In [ ]:
new_masky = ['category','num_ingredients','cups_sugar','cups_flour',
            'eggs','baking_soda',
            'baking_temp','water','milk','buttermilk','salt']
allrec_df2 = allrecipes_df[new_masky]
sns.pairplot(allrec_df2,hue ='category' )

## Revisit Logistic Regression with Smaller Feature set

In [ ]:
new_mask = ['num_ingredients','cups_sugar','cups_flour',
            'eggs','baking_soda',
            'baking_temp','water','milk','buttermilk','salt']


X_train_val_scaled_sh = X_train_val_scaled[new_mask]
X_train_scaled_sh = X_train_scaled[new_mask]
X_val_scaled_sh = X_val_scaled[new_mask]
X_test_scaled_sh = X_test_scaled[new_mask]

In [ ]:
logreg.fit(X_train_scaled_sh,y_train)
logreg.score(X_train_scaled_sh,y_train)

In [ ]:
logreg.score(X_val_scaled_sh,y_val)

In [ ]:
logreg_coeff.model_coefficient

In [ ]:
logreg_coeff = pd.DataFrame({'feature_name': logreg.feature_names_in_, 
                             'model_coefficient': logreg.coef_.transpose().flatten()})
logreg_coeff = logreg_coeff.sort_values(by = 'model_coefficient',ascending = False)
fig, ax = plt.subplots() # or 
fig.patch.set_facecolor('none')
ax.patch.set_facecolor('none')


sns.set_theme(style="whitegrid")
ax = sns.barplot(x="feature_name", y="model_coefficient", data=logreg_coeff,palette = 'coolwarm')
ax.set_title('Positive = Scones', fontsize = 15)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45,horizontalalignment = ('right'), fontsize = 14)
ax.set_xlabel('Feature Names', fontsize = 15)
ax.set_ylabel('Model Coefficients', fontsize = 15)
#ax.set_title('Positive = Scones')
fig.savefig('lasso_coefficients.pdf', format='eps',bbox_inches = 'tight')

In [ ]:
plot_roc_curve(logreg,X_train_scaled_sh,y_train)

In [ ]:
plot_roc_curve(logreg,X_val_scaled_sh,y_val)

## Revisit Random Forrest

In [ ]:
frst = RandomForestClassifier(n_estimators = 100, max_depth=3, random_state=0)
frst.fit(X_train_scaled_sh, y_train)

In [ ]:
frst.score(X_train_scaled_sh, y_train)

In [ ]:
frst.score(X_val_scaled_sh, y_val)

In [ ]:
plot_roc_curve(frst,X_train_scaled_sh,y_train)

In [ ]:
plot_roc_curve(frst,X_val_scaled_sh,y_val)

## Revist KNN

In [ ]:
n_neighbors = list(range(1,20))
scores =[]
for x in n_neighbors:
    neigh = KNeighborsClassifier(n_neighbors = x)
    neigh.fit(X_train_scaled_sh,y_train)
    scores.append(neigh.score(X_val_scaled_sh,y_val))
plt.plot(n_neighbors,scores)
plt.xlabel('number of neighbors')
plt.ylabel('validation score')

In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 8)
neigh.fit(X_train_scaled_sh,y_train)

In [ ]:
neigh.score(X_train_scaled_sh,y_train)

In [ ]:
neigh.score(X_val_scaled_sh,y_val)

In [ ]:
plot_roc_curve(neigh,X_train_scaled_sh,y_train)

In [ ]:
plot_roc_curve(neigh,X_val_scaled_sh,y_val)

## Ensembling

In [ ]:
#### Lots of code in this section is adapted from Metis course videos

In [ ]:
model_list = [('knn',neigh),
              ('forest',frst),
              ('log_reg',logreg)]

for model in model_list:
    curr_name,curr_model = model
    curr_model.fit(X_train_scaled_sh,y_train)
    curr_score = curr_model.score(X_train_scaled_sh,y_train)
    print(f'{curr_name} score: {curr_score}')

In [ ]:
model_list = [('knn',neigh),
              ('forest',frst),
              ('log_reg',logreg)]

for model in model_list:
    curr_name,curr_model = model
    curr_model.fit(X_train_scaled_sh,y_train)
    curr_score = curr_model.score(X_val_scaled_sh,y_val)
    print(f'{curr_name} score: {curr_score}')

In [ ]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators=model_list,
                      voting = 'hard',
                      weights = [1,1,1])
vc.fit(X_train_scaled_sh,y_train)
vc.score(X_val_scaled_sh,y_val)



In [ ]:
vc.score(X_train_scaled_sh,y_train)

In [ ]:
y_val_pred = vc.predict(X_val_scaled_sh)

## See what values are wrong

In [ ]:
### I want to see which values its getting wrong
results = pd.DataFrame(y_val-y_val_pred).sort_values(by = 'binary_cat')   

In [ ]:
misclassified_biscuits = y_val1[y_val1.index.isin(results[results.binary_cat == -1].index.tolist())]
misclassified_scones = y_val1[y_val1.index.isin(results[results.binary_cat == 1].index.tolist())]

In [ ]:
misclassified_biscuits

In [ ]:
misclassified_scones

## Run it on the test set

In [ ]:
vc.score(X_test_scaled_sh,y_test)


In [ ]:
neigh.score(X_test_scaled_sh,y_test)


In [ ]:
logreg.score(X_test_scaled_sh,y_test)

In [ ]:
frst.score(X_test_scaled_sh,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, vc.predict(X_test_scaled_sh))

In [ ]:
y_test_pred = vc.predict(X_test_scaled_sh)

In [ ]:
results = pd.DataFrame(y_test-y_test_pred).sort_values(by = 'binary_cat')   
misclassified_biscuits = y_test1[y_test1.index.isin(results[results.binary_cat == -1].index.tolist())]
misclassified_scones = y_test1[y_test1.index.isin(results[results.binary_cat == 1].index.tolist())]

In [ ]:
misclassified_biscuits

In [ ]:
misclassified_scones